In [1]:
objective = """
function my_custom_objective(tree, dataset::Dataset{T,L}, options) where {T,L} 
    penalty_term = L(0)
    #predd = L(1000)
    tot_features = 1:7
    concrete_contribution = L(100)
    web_reinf_contribution = L(100)
    tol = L(0.0000000000000001)

    if tree.degree != 2
        penalty_term += L(10000)
    else
        left = tree.l
        if left.degree != 2
            penalty_term += L(1000)
        else
            # vs
            fn = left.l
            features=[1,2] #function of only fck or ks
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vs, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vs) ? L(100) : L(0)
            
            
            # vt
            fn = left.r
            features=[3] #function of only rle
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vt, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vt) ? L(100) : L(0)

            vss = tol .+ vs .* transpose(dataset.X[:, 6])# vs * sin_theta
            vtt = tol .+ vt .* transpose(dataset.X[:, 7])# vt * tan_theta
            
            concrete_contribution = 1.0 ./(1.0 ./ vss + 1.0 ./ vtt)
        end
        right = tree.r
        if right.degree != 2
            penalty_term += L(1000)
        else
            # vh
            fn = right.l
            features=[4] #function of only rhe
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vh, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vh) ? L(100) : L(0)
            
            # vv
            fn = right.r
            features=[5] #function of only rve
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vv, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vv) ? L(100) : L(0)

            web_reinf_contribution = vh .* transpose(dataset.X[:, 7]) .+ vv   # vh * tan_theta +vv
        end
    end
    r = dataset.y ./(concrete_contribution .+ web_reinf_contribution) .- 1.0
    MAPE = sum(abs.(r)) ./ length(r)
    return ( penalty_term  + MAPE )
end

my_custom_objective
"""
from pysr import PySRRegressor
import pandas as pd
import numpy as np
PC  = pd.read_excel('data.xlsx',sheet_name='RCDBs')[['fck','av','b','wb','d','h','rf','rh','rv','fyt','fyh','fyv','Vexp']]

PC['rho']=PC['rf']
PC['rhe']=PC['rh']*PC['fyh']/PC['fck']
PC['rve']=PC['rv']*PC['fyv']/PC['fck']
PC['a_d']=PC['av']/PC['d']
PC['vn']=PC['Vexp']/(PC['d'] * PC['b'])/PC['fck']*1000

n_rho=42.6/PC['fck']**0.5*PC['rho']
k=(n_rho**2.0+2.0*n_rho)**0.5-n_rho
jd_a=(1.0-k/3.0)/PC['a_d']
theta = np.arctan(jd_a)
PC['t_th']=jd_a;PC['c_th']=np.cos(theta);PC['s_th']=np.sin(theta)
#PC['s1_t']=PC['s1']*jd_a #np.tan(theta)
PC['ks']=((k**2.0+(PC['wb']/PC['d'])**2.0)**0.5)
#PC['br']=(PC['rho']*PC['fyt']+0.5*PC['fck']**0.5*2.0*(PC['h']/PC['d']-1.0))/PC['fck']*jd_a#np.tan(theta)
PC['rle']=(PC['rho']*PC['fyt'])/PC['fck']

PC['fc']=PC['fck']/30.0
features=['ks','fc','rle','rhe','rve','s_th','t_th']

X=PC[features];y=PC['vn']

modelior11=[]
for j in range(5):
    #population_size is the equation size
    model7 = PySRRegressor(niterations=200,populations=20,population_size=50,maxsize=50,
                      nested_constraints={"^":{"^":1}},parsimony= 0.02,#adaptive_parsimony_scaling=1000,
                      constraints={"^":(-1,10)},
                      binary_operators=["+", "*","^"],#special(x,y)=x^2.0+y
                      denoise=True,loss_function=objective,model_selection ='accuracy',
                      )
    model7.fit(X,y)
    i=0
    modelior11.append(model7)
    while i <len(model7.equations_):
        print(model7.sympy(i))
        i=i+1

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Compiling Julia backend...


[ Info: Started!



Expressions evaluated per second: 3.590e+03
Head worker occupation: 7.7%
Progress: 12 / 800 total iterations (1.500%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = -0.45813
3           2.001e+03  8.045e-01  y = (rhe + rle)
5           1.001e+03  3.463e-01  y = (1.0923 + (rhe ^ rve))
7           1.037e+02  1.133e+00  y = ((fc + rle) + (rhe ^ rve))
9           4.299e+00  1.592e+00  y = (((ks * fc) + rle) + (rhe + rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.150e+03
Head worker occupation: 0.6%
Progress: 28 / 800 total iterations (3.500%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1  

...


Expressions evaluated per second: 1.560e+03
Head worker occupation: 8.3%
Progress: 4 / 800 total iterations (0.500%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = fc
3           2.001e+03  8.045e-01  y = (0.12078 * 0.38381)
5           1.001e+03  3.463e-01  y = (ks + (rhe * rve))
7           2.006e+02  8.038e-01  y = ((ks * rle) + (rhe * 0.30253))
9           1.010e+02  3.432e-01  y = ((ks ^ rle) * ((0.55467 ^ rhe) + rve))
11          1.008e+02  1.024e-03  y = ((fc + rle) ^ ((1.2336 + (1.2864 + rhe)) ^ rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 2.790e+03
Head worker occupation: 1.0%
Progress: 19 / 800 total iterations (2.375%)
Hall of Fame:
----------------------------------------------------

c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 0.000e+00
Head worker occupation: 16.3%
Progress: 10 / 800 total iterations (1.250%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = ks
3           2.001e+03  8.045e-01  y = (ks * 0.68995)
5           1.001e+03  3.463e-01  y = (0.32443 ^ (rhe ^ rve))
7           1.037e+02  1.133e+00  y = ((fc ^ rle) ^ (rhe + rve))
9           1.004e+02  1.651e-02  y = ((fc + rle) * (rhe + (0.11584 + rve)))
11          2.699e+00  1.808e+00  y = (((ks + fc) + (rle ^ 0.78365)) ^ (rhe * rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 3.650e+03
Head worker occupation: 9.8%
Progress: 14 / 800 total iterations (1.750%)
Hall of Fame:
---------------------------------------------------------

c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.580e+03
Head worker occupation: 19.2%
Progress: 15 / 800 total iterations (1.875%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = rle
3           2.001e+03  8.045e-01  y = (fc ^ 0.18141)
5           1.001e+03  3.463e-01  y = (rle ^ (rhe * rve))
7           1.037e+02  1.133e+00  y = ((fc ^ rle) + (rhe * rve))
9           3.551e+00  1.687e+00  y = (((ks + fc) * rle) * (rhe + rve))
11          9.762e-01  6.457e-01  y = (((ks + fc) * rle) * ((rhe ^ rhe) + rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 3.310e+03
Head worker occupation: 1.2%
Progress: 29 / 800 total iterations (3.625%)
Hall of Fame:
---------------------------------------------------------------------

In [1]:
objective = """
function my_custom_objective(tree, dataset::Dataset{T,L}, options) where {T,L} 
    penalty_term = L(0)
    #predd = L(1000)
    tot_features = 1:7
    concrete_contribution = L(100)
    web_reinf_contribution = L(100)
    tol = L(0.0000000000000001)

    if tree.degree != 2
        penalty_term += L(10000)
    else
        left = tree.l
        if left.degree != 2
            penalty_term += L(1000)
        else
            # vs
            fn = left.l
            features=[1,2] #function of only fck or ks
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vs, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vs) ? L(100) : L(0)
            
            
            # vt
            fn = left.r
            features=[3] #function of only rle
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vt, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vt) ? L(100) : L(0)

            vss = tol .+ vs .* transpose(dataset.X[:, 6])# vs * sin_theta
            vtt = tol .+ vt .* transpose(dataset.X[:, 7])# vt * tan_theta
            
            concrete_contribution = 1.0 ./(1.0 ./ vss + 1.0 ./ vtt)
        end
        right = tree.r
        if right.degree != 2
            penalty_term += L(1000)
        else
            # vh
            fn = right.l
            features=[4] #function of only rhe
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vh, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vh) ? L(100) : L(0)
            
            # vv
            fn = right.r
            features=[5] #function of only rve
            remain_feat = setdiff(tot_features, features)
            should_nt = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in remain_feat]
            should = [any(n -> n.degree == 0 && n.constant == false && n.feature == f, fn) for f in features]
            penalty_term += L(100) * (sum(should_nt)) + L(100) * (length(features) - sum(should))
            vv, flag = eval_tree_array(fn, dataset.X, options)
            if !flag
                return L(Inf)
            end
            penalty_term += any(x -> x < 0, vv) ? L(100) : L(0)

            web_reinf_contribution = vh .* transpose(dataset.X[:, 7]) .+ vv   # vh * tan_theta +vv
        end
    end
    r = dataset.y ./(concrete_contribution .+ web_reinf_contribution) .- 1.0
    MAPE = sum(abs.(r)) ./ length(r)
    return ( penalty_term  + MAPE )
end

my_custom_objective
"""
from pysr import PySRRegressor
import pandas as pd
import numpy as np
PC  = pd.read_excel('data.xlsx',sheet_name='RCCs')[['fck','av','b','wc','d','h','rf','rh','rv','fyt','fyh','fyv','Nc','Vexp']]

PC['rho']=PC['rf']-PC['Nc']/PC['fyt']/(PC['d'] * PC['b'])*1000
PC['rhe']=PC['rh']*PC['fyh']/PC['fck']
PC['rve']=PC['rv']*PC['fyv']/PC['fck']
PC['a_d']=PC['av']/PC['d']
PC['vn']=PC['Vexp']/(PC['d'] * PC['b'])/PC['fck']*1000

n_rho=42.6/PC['fck']**0.5*PC['rho']
k=(n_rho**2.0+2.0*n_rho)**0.5-n_rho
jd_a=(1.0-k/3.0)/PC['a_d']
theta = np.arctan(jd_a)
PC['t_th']=jd_a;PC['c_th']=np.cos(theta);PC['s_th']=np.sin(theta)
#PC['s1_t']=PC['s1']*jd_a #np.tan(theta)
PC['ks']=((k**2.0+(PC['wc']/2.0/PC['d'])**2.0)**0.5)
#PC['br']=(PC['rho']*PC['fyt']+0.5*PC['fck']**0.5*2.0*(PC['h']/PC['d']-1.0))/PC['fck']*jd_a#np.tan(theta)
PC['rle']=(PC['rho']*PC['fyt'])/PC['fck']

PC['fc']=PC['fck']/30.0
features=['ks','fc','rle','rhe','rve','s_th','t_th']

X=PC[features];y=PC['vn']

modelior767=[]
for j in range(5):
    #population_size is the equation size
    model7 = PySRRegressor(niterations=200,populations=20,population_size=50,maxsize=50,
                      nested_constraints={"^":{"^":1}},parsimony= 0.02,#adaptive_parsimony_scaling=1000,
                      constraints={"^":(-1,10)},
                      binary_operators=["+", "*","^"],#special(x,y)=x^2.0+y
                      denoise=True,loss_function=objective,model_selection ='accuracy',
                      )
    model7.fit(X,y)
    i=0
    modelior767.append(model7)
    while i <len(model7.equations_):
        print(model7.sympy(i))
        i=i+1

Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Compiling Julia backend...


[ Info: Started!



Expressions evaluated per second: 4.000e+03
Head worker occupation: 10.9%
Progress: 13 / 2000 total iterations (0.650%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = fc
3           2.001e+03  8.045e-01  y = (rhe * rve)
5           1.001e+03  3.463e-01  y = (rhe * (rhe * rve))
7           1.062e+02  1.122e+00  y = ((fc ^ rle) * (rhe ^ rve))
9           6.069e+00  1.431e+00  y = (((fc + ks) * rle) + (rhe + rve))
11          8.171e-01  1.003e+00  y = (((ks + fc) * rle) ^ ((0.64982 + rhe) ^ rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.110e+03
Head worker occupation: 1.7%
Progress: 29 / 2000 total iterations (1.450%)
Hall of Fame:
-------------------------------------------------------------------

...


Expressions evaluated per second: 2.860e+03
Head worker occupation: 6.0%
Progress: 5 / 2000 total iterations (0.250%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = 0.5685
3           2.001e+03  8.045e-01  y = (rhe * rle)
5           1.001e+03  3.463e-01  y = (-0.71466 * (rhe + rve))
7           1.062e+02  1.122e+00  y = ((fc + rle) ^ (rhe * rve))
9           6.069e+00  1.431e+00  y = (((ks + fc) + rle) * (rhe + rve))
11          4.581e+00  1.407e-01  y = (((ks + fc) * rle) ^ ((rhe ^ 0.57517) * rve))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 6.410e+03
Head worker occupation: 4.2%
Progress: 27 / 2000 total iterations (1.350%)
Hall of Fame:
------------------------------------------------------------

c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 3.910e+03
Head worker occupation: 64.6%. This is high, and will prevent efficient resource usage. Increase `ncyclesperiteration` to reduce load on head worker.
Progress: 11 / 2000 total iterations (0.550%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = fc
3           2.001e+03  8.045e-01  y = (0.46576 * 1.0372)
5           1.001e+03  3.463e-01  y = (0.71911 * (rhe + rve))
7           1.062e+02  1.122e+00  y = ((fc ^ rle) + (rhe + rve))
9           6.661e+00  1.385e+00  y = (((ks * fc) ^ rle) ^ (rhe + rve))
11          8.337e-01  1.039e+00  y = (((ks + fc) + rle) + (rhe ^ (rve ^ rve)))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.380e+03
Head worker occupation: 4.2%


c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1230: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\pysr\sr.py:1297: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\Users\osama\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 2.470e+03
Head worker occupation: 13.3%
Progress: 7 / 2000 total iterations (0.350%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.000e+04  1.594e+01  y = 0.45561
3           2.001e+03  8.045e-01  y = (rhe * rve)
5           1.001e+03  3.463e-01  y = (ks + (rhe * rve))
7           1.062e+02  1.122e+00  y = ((fc ^ rle) ^ (rhe * rve))
9           6.069e+00  1.431e+00  y = (((ks + fc) + rle) + (rhe ^ rve))
11          2.887e-01  1.523e+00  y = (((fc * ks) ^ rle) ^ (rhe * (rve + 0.14204)))
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 2.680e+03
Head worker occupation: 0.6%
Progress: 20 / 2000 total iterations (1.000%)
Hall of Fame:
----------------------------------------------------------------